In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchaudio
import torchaudio.transforms as transforms
import os 
import numpy as np
from tqdm import tqdm
from RSRTxReadBin.RTxReadBin import RTxReadBin
import pandas as pd

In [24]:
# Custom Dataset
class SpeechDataset(Dataset):
    def __init__(self, path = "waveforms_r6p8_fast/", file_list=[], ch=0, n_mels=64, n_fft=2048, hop_length=512):
        self.file_list = file_list
        self.ch = ch
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.path = path
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        waveform, sample_rate = self.load_waveform(file_path, self.ch)
        mel_spectrogram = self.transform_audio(waveform, sample_rate) # No use the spectrogram
        label = self.get_label_from_filename(file_path)
        return waveform, label

    def load_waveform(self, file_path, ch):
        wfm_data, b, meta_data = RTxReadBin(self.path+file_path, nNofChannels=2)
        wfm_data = np.array(wfm_data[:, 0, ch])[:int(len(wfm_data[:, 0, ch]) / 2)]
        sample_rate = int(1 / meta_data["Resolution"])
        waveform = torch.from_numpy(wfm_data).float()
        return waveform, sample_rate

    def transform_audio(self, waveform, sample_rate):
        transform = transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=self.n_mels, n_fft=self.n_fft, hop_length=self.hop_length)
        mel_spectrogram = transform(waveform.unsqueeze(0))
        return mel_spectrogram

    def get_label_from_filename(self, filename):
        return int(os.path.basename(filename).split('_')[0])

class SmallSpeechCNN(nn.Module):
    def __init__(self, num_classes):
        super(SmallSpeechCNN, self).__init__()
        self.conv1 = nn.Conv2d(batch_size, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=9, stride=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.fc1 = nn.Linear(64, 128, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        #print("Covn1", x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        #print("Pool", x.shape)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        #print("Adaptive Pool", x.shape)
        x = x.view(-1, 64)  # Adjusted input size for fc1
        #print("View", x.shape)
        x = F.relu(self.fc1(x))
        #print("FC1", x.shape)
        x = self.fc2(x)
        #print("FC2", x.shape)
        #x = self.softmax(x)
        return x
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=10, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        #print(x.shape)
        x = x.unsqueeze(0)
        #print(x.shape)
        x = self.conv1(x)
        #print(x.shape)
        x = F.relu(self.bn1(x))
        #print(x.shape)
        x = self.pool1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        #print(x.shape)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        #print(x.shape)
        x = x.squeeze(0)
        #print(x.shape)
        return F.log_softmax(x)
# Hyperparameters
num_classes = 10
num_epochs = 25
batch_size = 1
learning_rate = 0.005

In [21]:
def test_model(model, dataloader):
    model.eval()
    device = next(model.parameters()).device
    running_corrects = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
    accuracy = running_corrects.double() / len(dataloader.dataset)
    print(f'Accuracy: {accuracy:.4f}')
    return accuracy

sample = ["r2p3", "r6p7", "r6p8","r6p9","r1p9"]
arb = False


for m in sample:
    for arb in [True,False]:
    # load model
        if arb:
            channel = 1
            model = torch.load(f"CNN_filter_{m}_softmax_arb.pt", map_location=torch.device('cpu'))
            print(f"Running ARB {m}")
        else:
            channel = 0
            model = torch.load(f"CNN_filter_{m}_softmax.pt", map_location=torch.device('cpu'))
            print(f"Running MTJ {m}")

        path = f"waveforms_{m}_fast_test/"

        file_list = os.listdir(path)
        file_list = [file for file in file_list if file.endswith('.Wfm.bin')]
        dataset = SpeechDataset(path = path, file_list=file_list, ch=channel)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)

        test_model(model, dataloader)

Running ARB r2p3


c:\Users\lasse\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (64) may be set too high. Or, the value for `n_freqs` (1025) may be set too low.
  warnings.warn(


Accuracy: 0.8367
Running MTJ r2p3
Accuracy: 0.7767
Running ARB r6p7
Accuracy: 0.8233
Running MTJ r6p7
Accuracy: 0.8400
Running ARB r6p8
Accuracy: 0.6867
Running MTJ r6p8
Accuracy: 0.8100
Running ARB r6p9
Accuracy: 0.8833
Running MTJ r6p9
Accuracy: 0.8967
Running ARB r1p9
Accuracy: 0.8300
Running MTJ r1p9
Accuracy: 0.6267


In [25]:
sample = ["r2p3", "r6p7", "r6p8","r6p9","r1p9"]
arb = False


for m in sample:
    for arb in [True,False]:
    # load model
        if arb:
            channel = 1
            model = torch.load(f"CNN_no_filter_{m}_arb.pt", map_location=torch.device('cpu'))
            print(f"Running ARB {m}")
        else:
            channel = 0
            model = torch.load(f"CNN_no_filter_{m}.pt", map_location=torch.device('cpu'))
            print(f"Running MTJ {m}")

        path = f"waveforms_{m}_fast_test/"

        file_list = os.listdir(path)
        file_list = [file for file in file_list if file.endswith('.Wfm.bin')]
        dataset = SpeechDataset(path = path, file_list=file_list, ch=channel)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)

        test_model(model, dataloader)

Running ARB r2p3


c:\Users\lasse\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (64) may be set too high. Or, the value for `n_freqs` (1025) may be set too low.
  warnings.warn(
C:\Users\lasse\AppData\Local\Temp/ipykernel_24108/2981074007.py:106: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Accuracy: 0.7067
Running MTJ r2p3
Accuracy: 0.5467
Running ARB r6p7
Accuracy: 0.6967
Running MTJ r6p7
Accuracy: 0.4833
Running ARB r6p8
Accuracy: 0.7200
Running MTJ r6p8
Accuracy: 0.5467
Running ARB r6p9
Accuracy: 0.5933
Running MTJ r6p9
Accuracy: 0.5567
Running ARB r1p9
Accuracy: 0.5233
Running MTJ r1p9
Accuracy: 0.5367


note  
MTJ - ARB

# r6p8 no filter
0.5467 - 0.72000

# r6p8 #  filtered
0.8100-0.6867

# r1p9  filtered
0.6267 - 0.8300

# r2p3 filtered
0.7767 - 0.8367

In [32]:
55.7-59.3

-3.5999999999999943